In [3]:
import tensorflow as tf
import numpy as np
import os
from numpy import array
from numpy import shape
from PIL import Image
from sklearn.preprocessing import OneHotEncoder

In [2]:
ohe = OneHotEncoder(n_values=500)

TRAIN_DIR = "/home/banana/Machine_Learning/CNN/MsCeleb/MsCelebV1-DevSet1/aligned/"
TEST_DIR = "/home/banana/Machine_Learning/CNN/MsCeleb/MsCelebV1-DevSet2/aligned/"

train_folder_list = array(os.listdir(TRAIN_DIR))
test_folder_list = array(os.listdir(TEST_DIR))

train_data = []
train_label = []

for i in range(len(train_folder_list)):
    with open(TRAIN_DIR + train_folder_list[i], "rb") as file:
        img = Image.open(file)
        img.convert("RGB")
        img = img.resize((96,96))
        data = np.asarray(img)
        train_data.append(data)
        train_label.append(i)
        
test_data = []
test_label = []
test = []
for i in range(len(test_folder_list)):
    with open(TEST_DIR + test_folder_list[i], "rb") as file:
        img = Image.open(file)
        img.convert("RGB")
        img = img.resize((96,96))
        data = np.asarray(img)
        test_data.append(data)
        test_label.append(i)
        
train_data = np.array(train_data)
train_label = np.array(train_label)
train_label = ohe.fit_transform(train_label.reshape(-1, 1)).toarray()
test_data = np.array(test_data)
test_label = np.array(test_label)
test_label = ohe.fit_transform(test_label.reshape(-1, 1)).toarray()

print(shape(train_data))
print(shape(train_label))
print(shape(test_data))
print(shape(test_label))

(500, 96, 96, 3)
(500, 500)
(500, 96, 96, 3)
(500, 500)


In [3]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
# dropout (keep_prob) rate  0.7~0.5 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# input place holders
X = tf.placeholder(tf.float32, [None, 96,96,3])
Y = tf.placeholder(tf.float32, [None, 500])

In [4]:
# L1 ImgIn shape=(?, 96, 96, 3)
W1 = tf.Variable(tf.random_normal([3, 3, 3, 32], stddev=0.01))
#    Conv     -> (?, 96, 96, 32)
#    Pool     -> (?, 48, 48, 32)
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# L2 ImgIn shape=(?, 48, 48, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv      ->(?, 48, 48, 64)
#    Pool      ->(?, 24, 24, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# L3 ImgIn shape=(?, 24, 24, 64)
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
#    Conv      ->(?, 24, 24, 64)
#    Pool      ->(?, 12, 12, 128)
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

# L4 ImgIn shape=(?, 12, 12, 128)
W4 = tf.Variable(tf.random_normal([3, 3, 128, 128], stddev=0.01))
#    Conv      ->(?, 12, 12, 128)
#    Pool      ->(?, 6, 6, 128)
L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding='SAME')
L4 = tf.nn.relu(L4)
L4 = tf.nn.max_pool(L4, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

# FC
L4 = tf.reshape(L4, [-1,6*6*128])
W5 = tf.get_variable("W5", shape=[6*6*128,500],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([500]))
hypothesis = tf.matmul(L4,W5) + b

In [5]:
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, 
                                                              labels=train_label))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [6]:
with tf.Session() as sess:
    # Initialize
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epochs):
        avg_cost = 0
        for i in range(len(train_folder_list)):
            feed_dict = {X: train_data , Y : train_label, keep_prob : 0.7}
            c, _ = sess.run([cost,optimizer], feed_dict=feed_dict)
        print("Epoch = " , epoch , " cost = ", cost)
    # Test model
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('Accuracy:', sess.run(accuracy, feed_dict={
    X: test_data, Y: test_label, keep_prob: 1}))


KeyboardInterrupt: 